# Designing Actor Systems

## Starting Out

- Imagine giving the task to a group of people, dividing it up.
- Consider the group to be of very large size.
- Start with how people with different tasks will talk with each other.
- Consider these “people” to be easily replaceable.
- Draw a diagram with how the task will be split up, including communication lines

## Example: the Link Checker

Write an actor system which given a URL will recursively download the
content, extract links and follow them, bounded by a maximum depth; all links encountered shall be returned

### Plan of Action
* Write web client which turns a URL into a HTTP body asynchronously. We will be using ”com.ning” % ”async-http-client” % ”1.7.19”
* Write a Getter actor for processing the body.
* Write a Controller which spawns Getters for all links encountered.
* Write a Receptionist managing one Controller per request.